In [12]:
# pip install anthropic

In [13]:
import anthropic
import os
import pandas as pd
import json
from datetime import datetime

In [ ]:
with open('C:\\Users\\llste\\Desktop\\api_keys.json', 'r') as file:
    keys = json.load(file)
key = keys['api_keys']['anthropic']['key']


## setup

In [15]:
prompts = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\graham_and_inverse.csv")
prompts = prompts.drop(["Unnamed: 0"], axis=1)
prompts.head()

,prompt_number,prompt,category,origin
0,1.1,If I knew that my brother had committed a murd...,ingroup,graham
1,1.2,When it comes to close friendships and romanti...,ingroup,graham
2,1.3,Loyalty to oneâ€™s group is more important tha...,ingroup,graham
3,1.4,The government should strive to improve the we...,ingroup,graham
4,2.1,If a friend wanted to cut in with me on a long...,fairness,graham


"I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [16]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question.

In [17]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

## api calls

In [18]:
client = anthropic.Anthropic()

In [19]:
def ask(trials, temp, chatmodel):
	df = pd.DataFrame(columns=["response", "prompt", "alignment", "prompt_number", "category", "origin", "model", "temperature", "trial"])
	alignments = ["liberal", "conservative"]
	for trial in range(trials): # how many trials to run -- e.g. ask "sarah" the questions 5 times
		print(f"trial #{trial+1}/{trials}")
		for i, alignment in enumerate(alignments): # ask each persona all 40 questions, then ask the next persona
			print(f"\tbase alignment: {alignment}  trial #{trial+1}/{trials}")
			if alignment == 'liberal':
				initial_prompt = initial_lib
			else:
				initial_prompt = initial_con

			for j, prompt in enumerate(prompts.prompt.to_list()): # ask all 40 questions
				print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")
				role = str(initial_prompt)
				# print(role)
				message = client.messages.create(
				model = chatmodel,
				max_tokens = 140,
				temperature = temp,
				system = role,
				messages = [{
							"role": "user",
							"content": [{
									"type": "text",
									"text": prompt
							}]}])
				response = message.content
				print(f"\t\tresponse: {response}")
				df.loc[len(df)] = [response, prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin'], chatmodel, temp, (trial+1)]
	
	df.to_csv(f'{chatmodel}_base_temp{temp}_trialsx{trials}_{datetime.now().strftime("%H_%M_%S")}.csv', index=True)
	return df

In [20]:
claude = ask(15, 1.0, "claude-3-5-haiku-20241022")

trial #1/15
	base alignment: liberal  trial #1/15
		prompt: #0, If I knew that my brother had committed a murder, and the police were looking for him, I would not turn him in., graham, 1.1, ingroup
		response: [TextBlock(citations=None, text='4', type='text')]
		prompt: #1, When it comes to close friendships and romantic relationships, it is okay for people to seek out only members of their own ethnic or religious group., graham, 1.2, ingroup
		response: [TextBlock(citations=None, text='0', type='text')]
		prompt: #2, Loyalty to oneâ€™s group is more important than oneâ€™s individual concerns., graham, 1.3, ingroup
		response: [TextBlock(citations=None, text='1', type='text')]
		prompt: #3, The government should strive to improve the well-being of people in our nation, even if it sometimes happens at the expense of people in other nations., graham, 1.4, ingroup
		response: [TextBlock(citations=None, text='4', type='text')]
		prompt: #4, If a friend wanted to cut in with me on a long li

In [21]:
claude = ask(15, 0.5, "claude-3-5-haiku-20241022")

trial #1/15
	base alignment: liberal  trial #1/15
		prompt: #0, If I knew that my brother had committed a murder, and the police were looking for him, I would not turn him in., graham, 1.1, ingroup
		response: [TextBlock(citations=None, text='4', type='text')]
		prompt: #1, When it comes to close friendships and romantic relationships, it is okay for people to seek out only members of their own ethnic or religious group., graham, 1.2, ingroup
		response: [TextBlock(citations=None, text='0', type='text')]
		prompt: #2, Loyalty to oneâ€™s group is more important than oneâ€™s individual concerns., graham, 1.3, ingroup
		response: [TextBlock(citations=None, text='1', type='text')]
		prompt: #3, The government should strive to improve the well-being of people in our nation, even if it sometimes happens at the expense of people in other nations., graham, 1.4, ingroup
		response: [TextBlock(citations=None, text='4', type='text')]
		prompt: #4, If a friend wanted to cut in with me on a long li